# CNN Transfer&Train&Predict

# Init Config Setup

In [7]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00004"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

# get workflow version info
resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/')
data = json.loads(resp.json())

# get Active workflow version
wf_ver_id = 0
data = sorted(data, key=lambda k: k['pk'])
for config in data:
    if config["fields"]["active_flag"] == "Y":
        wf_ver_id = config["pk"]

wf_ver_id = str(wf_ver_id)

print("Network ID="+nn_id)
print("Workflow Version="+wf_ver_id)
print("")

def spaceprint(val, cnt):
    leng = len(str(val))
    cnt = cnt - leng
    restr = ""
    for i in range(cnt):
        restr += " "
    restr = restr+str(val)
    return restr

node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "param":{"epoch":20
                                  ,"traincnt": 5,
                                     "batch_size":10000,
                                     "predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                     "layeroutputs":32,
                                     "type":"category"
                                     }
                         ,"layer1": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                          ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
        ,"labels":[]
                        })
netconf = json.loads(resp.json())
print("Network Config : {0}".format(netconf))
print("")

node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         

                     })
dataconf = json.loads(resp.json())
print("Train Data Config : {0}".format(dataconf))
print("")

node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("Eval Config : {0}".format(evalconf))
print("")

node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}

                     })
edataconf = json.loads(resp.json())
print("Eval Data Config : {0}".format(edataconf))
print("")

Network ID=nn00004
Workflow Version=18

Network Config : {'layer1': {'cnnfilter': [3, 3], 'maxpoolstride': [2, 2], 'maxpoolmatrix': [2, 2], 'cnnstride': [1, 1], 'padding': 'SAME', 'layercnt': 2, 'active': 'relu', 'droprate': '0.8', 'type': 'cnn'}, 'layer2': {'cnnfilter': [3, 3], 'maxpoolstride': [2, 2], 'maxpoolmatrix': [2, 2], 'cnnstride': [1, 1], 'padding': 'SAME', 'layercnt': 1, 'active': 'relu', 'droprate': '0.8', 'type': 'cnn'}, 'out': {'node_out': 625, 'active': 'softmax', 'padding': 'SAME'}, 'param': {'epoch': 20, 'traincnt': 5, 'batch_size': 10000, 'predictcnt': 10}, 'labels': [], 'config': {'learnrate': 0.001, 'layeroutputs': 32, 'type': 'category', 'num_classes': 10}}

Train Data Config : {'store_path': '/hoya_str_root/nn00004/18/datasrc', 'preprocess': {'channel': 3, 'y_size': 32, 'x_size': 32}, 'source_path': '/hoya_src_root/nn00004/18/datasrc'}

Eval Config : {}

Eval Data Config : {'store_path': '/hoya_str_root/nn00004/18/evaldata', 'preprocess': {'channel': 3, 'y_size': 

# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

아래 API의 경우 Test Data인지 Train Data인지 선택할수 있는 Node 정보를 모두 가져와 준다.

In [8]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

get file node list
{'nn_wf_node_desc': 'Train Data Node', 'nn_wf_node_id': 'nn00004_18_datasrc', 'nn_wf_node_name': 'datasrc'}
{'nn_wf_node_desc': 'Evaluation Data Node', 'nn_wf_node_id': 'nn00004_18_evaldata', 'nn_wf_node_name': 'evaldata'}



아래 API는 한개의 파일을 Train Folder & Test Folder 에 넣어준다.

In [10]:
# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

create file node list
{'File0': {'File': 'File Create.(/hoya_src_root/nn00004/18/datasrc/sample_cnn_img.zip1)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00004/18/datasrc/sample_cnn_img.zip1)'}}
{'File0': {'File': 'File Create.(/hoya_src_root/nn00004/18/evaldata/sample_cnn_img.zip1)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00004/18/evaldata/sample_cnn_img.zip1)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br> 

In [11]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result":
        try:
            for tr in train["TrainResult"]:
                print(tr)
        except:
            maxcnt = 0
            line = ""
            for label in train["labels"]:
                if maxcnt<len(label):
                    maxcnt = len(label)

            for i in range(len(train["labels"])):
                for j in range(maxcnt+4):
                    line += "="

            label_sub = []
            for label in train["labels"]:
                label = spaceprint(label,maxcnt)
                label_sub.append(label)

            space = ""
            for s in range(maxcnt):
                space +=" "

            print(space, label_sub)
            print(space, line)
            for i in range(len(train["labels"])):
                predict_sub = []
                for j in range(len(train["predicts"][i])):
                    pred = spaceprint(train["predicts"][i][j],maxcnt)

                    predict_sub.append(pred)

                print(spaceprint(train["labels"][i],maxcnt), predict_sub)

['Trainning ..................................................']
['Global Step:     15, Training Batch Accuracy:  11.76%']

['Trainning ..................................................']
['Global Step:     20, Training Batch Accuracy:  29.41%']

['Trainning ..................................................']
['Global Step:     25, Training Batch Accuracy:  29.41%']

['Trainning ..................................................']
['Global Step:     30, Training Batch Accuracy:  11.76%']

['Trainning ..................................................']
['Global Step:     35, Training Batch Accuracy:  52.94%']

['Trainning ..................................................']
['Global Step:     40, Training Batch Accuracy:  64.71%']

['Trainning ..................................................']
['Global Step:     45, Training Batch Accuracy:  52.94%']

['Trainning ..................................................']
['Global Step:     50, Training Batch Accuracy:  70.59%']

['Trainn

# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [5]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')

FileName = 2car.jpg
['glovemarketabcde', 'dog', 'glove', 'car', 'motor', 'airplane', 'ca    t', 'cat', 'bolt']
[871.95508, 39.54843, -23.13018, -59.26953, -81.88757, -133.88457, -227.80824, -234.73695, -479.1026]

FileName = 1bolt.jpg
['glovemarketabcde', 'dog', 'car', 'glove', 'motor', 'airplane', 'cat', 'ca    t', 'bolt']
[1197.63708, 79.00513, -22.77024, -37.27081, -84.71973, -212.42955, -270.82303, -276.75476, -647.57831]

FileName = 1glove.jpg
['glovemarketabcde', 'dog', 'glove', 'car', 'motor', 'airplane', 'ca    t', 'cat', 'bolt']
[1340.97681, 62.86176, 31.42587, 16.51012, -68.01265, -233.03719, -318.09222, -321.92133, -755.56445]

FileName = 1car.jpg
['glovemarketabcde', 'glove', 'dog', 'car', 'motor', 'airplane', 'ca    t', 'cat', 'bolt']
[858.25964, 55.42708, 49.4529, 15.50914, -85.92107, -146.89845, -169.56615, -205.61496, -450.46146]

FileName = 1air.jpg
['glovemarketabcde', 'dog', 'glove', 'car', 'motor', 'airplane', 'ca    t', 'cat', 'bolt']
[934.37628, 28.74143, 9.06982,